In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

measuring the impurity in the data set 

In [ ]:
def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    prob_sq = (counts / counts.sum()) ** 2
    return 1 - prob_sq.sum() 

spliting the data bases on the given thresshold 

In [ ]:
def split_dataset(X, y, feature, threshold):
    left_indices = X[:, feature] <= threshold
    right_indices = X[:, feature] > threshold
    return (X[left_indices], y[left_indices]), (X[right_indices], y[right_indices])

obtaining the best thresshold to split

In [ ]:
def best_split(X, y):
    best_feature, best_threshold = None, None
    best_gini = float('inf')
    best_splits = None

    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            (X_left, y_left), (X_right, y_right) = split_dataset(X, y, feature, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            gini_left = gini_impurity(y_left)
            gini_right = gini_impurity(y_right)
            gini_split = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

            if gini_split < best_gini:
                best_gini = gini_split
                best_feature = feature
                best_threshold = threshold
                best_splits = (X_left, y_left, X_right, y_right)

    return best_feature, best_threshold, best_splits

building the decision tree

In [ ]:
def build_tree(X, y, max_depth=None, min_samples_split=2, depth=0):
    
    #stopping condition 
    if (max_depth is not None and depth >= max_depth) or len(y) < min_samples_split or len(np.unique(y)) == 1:
        return np.argmax(np.bincount(y))
    
    feature, threshold, splits = best_split(X, y)
    if splits is None:
        return np.argmax(np.bincount(y))

    X_left, y_left, X_right, y_right = splits
    
    #sub branches
    left_subtree = build_tree(X_left, y_left, max_depth, min_samples_split, depth + 1)
    right_subtree = build_tree(X_right, y_right, max_depth, min_samples_split, depth + 1)

    return {"feature": feature, "threshold": threshold, "left": left_subtree, "right": right_subtree}

predicting the new data set 

In [ ]:
def predict_sample(x, tree):
    if not isinstance(tree, dict):
        return tree

    feature = tree["feature"]
    threshold = tree["threshold"]

    if x[feature] <= threshold:
        return predict_sample(x, tree["left"])
    else:
        return predict_sample(x, tree["right"])

def predict(X, tree):
    return np.array([predict_sample(x, tree) for x in X])

metric calculation

In [ ]:
def calculate_f1_score(y_true, y_pred):
    classes = np.unique(y_true)
    f1_scores = []

    for cls in classes:
        tp = np.sum((y_pred == cls) & (y_true == cls))
        fp = np.sum((y_pred == cls) & (y_true != cls))
        fn = np.sum((y_pred != cls) & (y_true == cls))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

    return np.mean(f1_scores)

In [ ]:
file_path = r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\multi_classification_train.csv"
data = pd.read_csv(file_path)

X = data.drop(columns=['ID', 'Class']).values
y = data['Class'].values

np.random.seed(42)
indices = np.random.permutation(len(X))
train_size = int(0.8 * len(X))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

X_train_mean = X_train.mean()
X_train_std  = X_train.std()
X_train = (X_train - X_train_mean)/X_train_std
X_test  = (X_test - X_train_mean)/X_train_std 
   
tree = build_tree(X_train, y_train, max_depth=10, min_samples_split=4)

y_train_pred = predict(X_train , tree)    
y_test_pred = predict(X_test, tree)

accuracy_test = np.mean(y_test_pred == y_test)
accuracy_train = np.mean(y_train_pred == y_train)
print(f"Test Accuracy: {accuracy_test:.2f} Train Accuracy: {accuracy_train:.2f}")
print(tree["feature"])
f1_score_test = calculate_f1_score(y_test, y_test_pred)
f1_score_train = calculate_f1_score(y_train, y_train_pred)
print(f"Test F1 Score: {f1_score_test:.2f} Train F1 Score: {f1_score_train:.2f}")

  

testing the data set 

In [ ]:
test_data = pd.read_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\multi_classification_test.csv")

X_new = test_data.drop(columns=["ID"]).values

X_new = (X_new - X_train_mean)/X_train_std 

y_new_pred = predict(X_new, tree)

test_data['Predicted'] = y_new_pred

test_data.to_csv(r"C:\Users\91944\OneDrive - Indian Institute of Technology Indian School of Mines Dhanbad\Desktop\ml\multi_classification_test_pred.csv", index=False)

print("Predictions saved in the original test data file.")
